## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,48.31,66,100,27.63,overcast clouds
1,1,Vestmannaeyjar,IS,63.4427,-20.2734,42.87,88,100,21.14,overcast clouds
2,2,Mataura,NZ,-46.1927,168.8643,68.49,52,55,4.76,broken clouds
3,3,Antalaha,MG,-14.9003,50.2788,76.53,88,100,4.12,overcast clouds
4,4,Andenes,NO,69.3143,16.1194,15.78,92,12,6.91,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Mataura,NZ,-46.1927,168.8643,68.49,52,55,4.76,broken clouds
3,3,Antalaha,MG,-14.9003,50.2788,76.53,88,100,4.12,overcast clouds
5,5,Airai,TL,-8.9266,125.4092,71.11,75,48,3.29,scattered clouds
7,7,Encruzilhada,BR,-15.5314,-40.9094,68.14,98,100,1.97,overcast clouds
8,8,Hithadhoo,MV,-0.6000,73.0833,82.00,71,26,6.11,scattered clouds
9,9,Kapaa,US,22.0752,-159.3190,77.32,70,0,1.99,clear sky
10,10,Rikitea,PF,-23.1203,-134.9692,77.85,79,100,20.49,overcast clouds
11,11,Vaini,TO,-21.2000,-175.2000,77.16,94,100,8.05,light rain
12,12,Vao,NC,-22.6667,167.4833,77.52,77,0,21.12,clear sky
13,13,Port Elizabeth,ZA,-33.9180,25.5701,71.10,83,0,5.75,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
np.where(pd.isnull(preferred_cities_df))

(array([ 91, 116, 162, 293, 311, 332, 333], dtype=int64),
 array([2, 2, 2, 2, 2, 2, 2], dtype=int64))

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Mataura,NZ,-46.1927,168.8643,68.49,52,55,4.76,broken clouds
3,3,Antalaha,MG,-14.9003,50.2788,76.53,88,100,4.12,overcast clouds
5,5,Airai,TL,-8.9266,125.4092,71.11,75,48,3.29,scattered clouds
7,7,Encruzilhada,BR,-15.5314,-40.9094,68.14,98,100,1.97,overcast clouds
8,8,Hithadhoo,MV,-0.6000,73.0833,82.00,71,26,6.11,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
696,696,Kovur,IN,14.4833,79.9833,79.66,66,15,7.38,few clouds
701,701,Mtambile,TZ,-5.3833,39.7000,79.97,80,59,12.37,broken clouds
702,702,Trelew,AR,-43.2490,-65.3051,67.98,49,7,6.49,clear sky
705,705,Mogadishu,SO,2.0371,45.3438,77.00,78,51,19.39,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Mataura,NZ,68.49,broken clouds,-46.1927,168.8643,
3,Antalaha,MG,76.53,overcast clouds,-14.9003,50.2788,
5,Airai,TL,71.11,scattered clouds,-8.9266,125.4092,
7,Encruzilhada,BR,68.14,overcast clouds,-15.5314,-40.9094,
8,Hithadhoo,MV,82.00,scattered clouds,-0.6000,73.0833,
9,Kapaa,US,77.32,clear sky,22.0752,-159.3190,
10,Rikitea,PF,77.85,overcast clouds,-23.1203,-134.9692,
11,Vaini,TO,77.16,light rain,-21.2000,-175.2000,
12,Vao,NC,77.52,clear sky,-22.6667,167.4833,
13,Port Elizabeth,ZA,71.10,clear sky,-33.9180,25.5701,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'])
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Mataura,NZ,68.49,broken clouds,-46.1927,168.8643,Ellie's Villa
3,Antalaha,MG,76.53,overcast clouds,-14.9003,50.2788,Hotel Vitasoa
7,Encruzilhada,BR,68.14,overcast clouds,-15.5314,-40.9094,Hotel e Restaurante Rio Pardo
8,Hithadhoo,MV,82.00,scattered clouds,-0.6000,73.0833,Scoop Guest House
9,Kapaa,US,77.32,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
695,Faaa,PF,80.56,broken clouds,-17.5500,-149.6000,Tahiti Nui Hotel
696,Kovur,IN,79.66,few clouds,14.4833,79.9833,Hotel Yesh Park
702,Trelew,AR,67.98,clear sky,-43.2490,-65.3051,Hotel Touring Club
705,Mogadishu,SO,77.00,broken clouds,2.0371,45.3438,Hotel Juba


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

In [13]:
fig

Figure(layout=FigureLayout(height='420px'))